In [1]:
# %matplotlib inline

In [20]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [22]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Zа-яёъА-ЯЁЪ.!?]+", r" ", s)
    return s

In [23]:
def load_opensubtitles_data(file_path_ru, file_path_en, num_samples=100_000):
    input_texts = []
    with open(file_path_en, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[:num_samples]:
            en_text = line.strip()
            input_texts.append(normalizeString(en_text))
    target_texts = []
    with open(file_path_ru, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[:num_samples]:
            ru_text = line.strip()
            target_texts.append(normalizeString(ru_text))
    return input_texts, target_texts

def load_anki_data(file_path, num_samples=100_000):
    input_texts = []
    target_texts = []
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[:num_samples]:
            en_text, ru_text = line.strip().split('\t')[:-1]
            input_texts.append(normalizeString(en_text))
            target_texts.append(normalizeString(ru_text)) 
    return input_texts, target_texts

In [24]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # lines = open('anki_data/rus.txt', encoding='utf-8').\
    #     read().strip().split('\n')

    # pairs = [[normalizeString(s) for s in l.split('\t')][0:2] for l in lines]
    anki_input_texts, anki_target_texts = load_anki_data('anki_data/rus.txt')
    opensub_input_texts, opensub_target_texts = load_opensubtitles_data('opensubtitles_data/OpenSubtitles.en-ru.ru',
                                                                 'opensubtitles_data/OpenSubtitles.en-ru.en')
    pairs = list(zip(anki_input_texts, anki_target_texts)) + list(zip(opensub_input_texts, opensub_target_texts))
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [25]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p, n=MAX_LENGTH ):
    return len(p[0].split(' ')) < n and \
        len(p[1].split(' ')) < n and \
        p[0].startswith(eng_prefixes)


def filterPairs(pairs, n=MAX_LENGTH):
    return [pair for pair in pairs if filterPair(pair,n)]

In [26]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)[:10000]
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'rus', False)
print(random.choice(pairs))

Reading lines...
Read 123924 sentence pairs
Trimmed to 10000 sentence pairs
Counting words...
Counted words:
eng 2035
rus 4361
('i m trying to quit .', 'я пытаюсь бросить .')


The Encoder
-----------





In [27]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
  
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------




In [28]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.wh = nn.Linear(self.hidden_size,self.hidden_size,bias=False)
        self.we = nn.Linear(self.hidden_size,self.hidden_size,bias=False)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size,bias=False)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        self.weight = nn.Parameter(torch.FloatTensor(1,1, self.hidden_size))
        

    def forward(self, input, hidden, encoder_outputs):
        # [1,1] [1,1,512] [10,512]
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        
        attn  = torch.tanh(self.wh(hidden)+self.we(encoder_outputs.unsqueeze(0)))
        scores = torch.bmm(self.weight,attn.transpose(1,2))
        weights = F.log_softmax(scores,2)
        
        context = torch.bmm(weights,encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded, context), 2)
        output = self.attn_combine(output)
                           
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [29]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [30]:
def train(input_tensor, target_tensor, encoder, decoder, 
          encoder_optimizer, decoder_optimizer, 
          criterion, max_length=MAX_LENGTH,teacher_forcing_ratio = 0.5):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()
    kc =5 + 100*(1-teacher_forcing_ratio)
    torch.nn.utils.clip_grad_norm_(encoder.parameters(), kc)
    torch.nn.utils.clip_grad_norm_(decoder.parameters(), kc)
                                   
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [31]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [32]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, 
               learning_rate=5e-5):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, 
                     criterion,teacher_forcing_ratio = 1-iter/n_iters)
        print_loss_total += loss
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            if print_loss_avg < 0.01:
                continue


In [15]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(
                input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [16]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [17]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)



In [18]:
trainIters(encoder1, attn_decoder1, 100_000, print_every=5000)

0m 24s (- 40m 14s) (1000 1%) 5.5383
0m 48s (- 39m 39s) (2000 2%) 4.4987
1m 18s (- 42m 2s) (3000 3%) 4.3849
1m 57s (- 46m 59s) (4000 4%) 4.2219
2m 38s (- 50m 4s) (5000 5%) 4.2636
3m 19s (- 52m 3s) (6000 6%) 4.2065
3m 55s (- 52m 6s) (7000 7%) 4.1659
4m 23s (- 50m 25s) (8000 8%) 4.1212
5m 5s (- 51m 27s) (9000 9%) 4.0816
5m 48s (- 52m 13s) (10000 10%) 4.0752
6m 32s (- 52m 52s) (11000 11%) 4.0398
7m 17s (- 53m 27s) (12000 12%) 3.9903
8m 2s (- 53m 51s) (13000 13%) 3.9323
8m 48s (- 54m 6s) (14000 14%) 3.8942
9m 25s (- 53m 24s) (15000 15%) 3.9311
9m 53s (- 51m 57s) (16000 16%) 3.8314
10m 20s (- 50m 30s) (17000 17%) 3.8174
10m 47s (- 49m 8s) (18000 18%) 3.8966
11m 13s (- 47m 51s) (19000 19%) 3.8317
11m 39s (- 46m 36s) (20000 20%) 3.7398
12m 5s (- 45m 29s) (21000 21%) 3.8329
12m 31s (- 44m 22s) (22000 22%) 3.8312
12m 55s (- 43m 16s) (23000 23%) 3.7666
13m 19s (- 42m 12s) (24000 24%) 3.6756
13m 44s (- 41m 13s) (25000 25%) 3.7493
14m 8s (- 40m 15s) (26000 26%) 3.6991
14m 33s (- 39m 20s) (27000 27%

In [19]:
evaluateRandomly(encoder1, attn_decoder1,20)

> i m a stuntman .
= я каскадер .
< я я я я я я я я я я

> we re too close .
= мы слишком близко .
< <EOS>

> i m still in boston .
= я все еще в бостоне .
< . . . . . . . . . .

> i m fearless .
= я бесстрашныи .
< . . . . . . . . . .

> they re in love .
= они влюблены .
< . . . . . . . . . .

> we re being called .
= нас вызывают .
< мы . . . . . . . . .

> they are all happy .
= они все счастливы .
< они они они они они они они они они они

> you re not bleeding .
= у тебя не течет кровь .
< <EOS>

> they re dead .
= они мертвы .
< они они они они они они они они они они

> i m almost ready .
= я почти готов .
< почти почти почти почти почти почти почти почти почти почти

> i m in a good mood .
= у меня хорошее настроение .
< . . . . . . . . . .

> i m not hungry now .
= я сеичас есть не хочу .
< . . . . . . . . . .

> i m in need of help .
= мне требуется помощь .
< . . . . . . . . . .

> i m dying .
= я умираю .
< я я я я я я я я я я

> you are beautiful .
= вы красивые .
< <EOS>